# Day2----一些核心的机器学习算法
在这个部分，我们会试着学习几种核心的算法，并加以不同的例子，数据来进行实践：
- Linear regression 线性回归
- Classification 分类
- Clustering 聚类
- Hidden Markov Model 隐藏马尔可夫模型

## Linear Regression 线性回归


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

x = [1,2,2.5,3,4]
y = [1,4,7,9,15]
plt.plot(x,y,'ro')  # 用 x y 来画点，'ro'是 参数 红色圆点
plt.axis([0,6,0,20]) # 指定 x y 轴数据的范围

可以看出，上面的点具有线性相关（ linear coorespondence ）
具有 **y = mx +b**的 近似


In [ ]:
plt.plot(x,y,'ro')
plt.axis([0,6,0,20])
plt.plot(np.unique(x), np.poly1d(np.polyfit(x,y,1))(np.unique(x)))
plt.show()


画出最佳线性拟合 （ line of best fit ）

### setup and import 设置及导入



In [ ]:
!pip install -q scikit-learn

In [ ]:
!pip install tensorflow 

In [ ]:
%tensorflow_version 2.x  # 只有使用google colab 的时候需要这个

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from IPython.display import clear_output
from six.moves import urllib

import tensorflow as tf
import tensorflow.compat.v2.feature_column as fc


### Data 数据的导入
这里使用的是泰坦尼克号数据集
它会包含乘客的一些信息：性别、年龄、船舱等级..... 以及最后是否存活的信息

In [ ]:
# Load dataset.

# 使用 pd 来读取 csv 文件
dftrain = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/train.csv') # training data
dfeval = pd.read_csv('https://storage.googleapis.com/tf-datasets/titanic/eval.csv') # testing data
# head()会返回最开始的5个entry
print(dftrain.head())

# pop 会把一个元素给分离出来，也就是这里的survived，这个东西会被用来当作label 
y_train = dftrain.pop('survived')
y_eval = dfeval.pop('survived')

print(dftrain.head())

In [ ]:
dftrain.age.hist(bins=20)


### Training vs Testing Data 训练数据和测试数据
传统来说，机器学习需要两组数据，训练集 （training data） 和 测试集（testing data） 

使用训练集来 feed 我们的模型，再用测试集来评估它的准确度。

再测试的时候，必须使用它之前没有见过的数据才行，所以我们需要测试集

### Feature Column 特征列
数据有两种类型： 数量型（Numeric） 和 种类型 （Categorical）

但是在训练的时候，我们需要把种类型也转换成数量型（eg. male =1, female =2）

TensorFlow 可以为我们代劳：


In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'n_siblings_spouses', 'parch', 'class', 'deck',
                       'embark_town', 'alone']
NUMERIC_COLUMNS = ['age', 'fare']

feature_columns = [] # 创建一个空的list
# 遍历 种类型 的 特征
for feature_name in CATEGORICAL_COLUMNS:
  # 获取 种类型 的所有不同种类有哪些
  vocabulary = dftrain[feature_name].unique()  # gets a list of all unique values from given feature column
  # 之后再把这些种类 append 到它的后面, 调用某个函数处理
  feature_columns.append(tf.feature_column.categorical_column_with_vocabulary_list(feature_name, vocabulary))

for feature_name in NUMERIC_COLUMNS:
  # 对应数量型的特征，不需要 vocabulary，直接调用另一个函数
  feature_columns.append(tf.feature_column.numeric_column(feature_name, dtype=tf.float32))

print(feature_columns)

In [ ]:
dftrain["sex"].unique()

### The Training Process 训练过程
显然，训练模型就是一个把数据喂（feed）给模型的过程。 在这里，泰坦尼克号的数据集只有600 多个数据，也许可以直接全部放到你的 RAM 上面。

但是如果数据很庞大，没有RAM能处理这样子大的数据，或者也会极度缓慢。

所以，通常的办法是分批次喂。 （batch）

在这里，我们一次喂32个数据 （batch-size）

We will actually feed these batches to our model multiple times acccording to the number of **epochs** （最大训练数）

Epochs is simply one stream of data of our entire dataset. 

The number of epochs we defineis the amount of times our model will see the entire dataset.

一个epochs 就是数据库的全部数据被模型读取一遍。
epochs 定义了我们的模型会读取整个数据库数据 多少次
也就是同一个数据，会被模型读取几遍 

每一次epochs， 改变数据输入的顺序，也许会慢慢得到更加准确的结果。

但是，如果次数太大，可能会有过拟合（overfitting）的情况。
这意味着模型开始记住这些点的位置。

在这个数据集上，它预测得很好，但是对于崭新的数据，预测效果不佳

### Input Function 输入方程
TensorFlow 模型 要求输入的模型必须是 tf.data.Dataset 类的 object。 所以我们需要一个input function 把现在的 pandas dataframe 转化成 那种 object

以下是一个例子

In [ ]:

def make_input_fn(data_df, label_df, num_epochs=10, shuffle=True, batch_size=32):
  # 这个函数，返回一个input 函数
  def input_function():  # 定义一个 inner function，这个就是我们需要的 input function
    ds = tf.data.Dataset.from_tensor_slices((dict(data_df), label_df))  # create tf.data.Dataset object with data and its label
    if shuffle:
      ds = ds.shuffle(1000)  # randomize order of data
    ds = ds.batch(batch_size).repeat(num_epochs)  # split dataset into batches of 32 and repeat process for number of epochs
    return ds  # return a batch of the dataset
  return input_function  # return a function object for use

# 给训练集和测试集分别创造 输入函数 
train_input_fn = make_input_fn(dftrain, y_train)  # here we will call the input_function that was returned to us to get a dataset object we can feed to the model
eval_input_fn = make_input_fn(dfeval, y_eval, num_epochs=1, shuffle=False)


### Creating the Model 建立模型

下面的linear_est 就是我们的模型本体

In [ ]:
pip install tensorflow-estimator

In [ ]:
linear_est = tf.estimator.LinearClassifier(feature_columns=feature_columns)
# 把前面定义的 feature_columns 传递给这个函数作为参数
# We create a linear estimtor by passing the feature columns we created earlier

### Training the model 训练模型


In [ ]:
linear_est.train(train_input_fn)  # 把训练集的 输入函数 传递给模型的 train（）函数
result = linear_est.evaluate(eval_input_fn)  # 把测试集的 输入函数 传递给 模型的evaluate（）函数 result 就是我们的结果

clear_output()  # clears console output
print(result['accuracy'])  # 打印结果的accuracy

## 崩溃了，搞了半天 发现在本地的环境里面就是不能运行 tensorflow-estimator